<a href="https://colab.research.google.com/github/MHDBST/BERT_examples/blob/master/MyBert_paragraph_document_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd
!pip install bert-tensorflow

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

('TPU address is', 'grpc://10.115.116.130:8470')
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 9481399364910338980),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14028473041940047913),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3802996930569954008),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8563387275645230877),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 11547112153883416950),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 14028010218007311323),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 10351207603824214348),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 8043261822612797081),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 157

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from bert import modeling
# import optimization
# import run_classifier
from bert import run_classifier_with_tfhub
# import tokenization

# import tfhub 
import tensorflow_hub as hub

W0520 03:40:10.135040 139864273921920 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
BUCKET = 'bert_example' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/combined/weighted_loss/smallBERT-parDoc_256'.format(BUCKET)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Model output directory: gs://bert_example/bert-tfhub/models/combined/weighted_loss/smallBERT-parDoc_256 *****


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1" ##Small Bert
# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1" ##Big Bert
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0520 03:40:21.492311 139864273921920 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0520 03:40:24.923955 139864273921920 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 10.0
MAX_SEQ_LENGTH = 256
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 200
SAVE_SUMMARY_STEPS = 100


In [0]:
from tensorflow import keras
import os
import re
import pandas as pd

# directory = '/content/alldata_3Dec_7Dec_PS_reindex_train_v3.csv'
# data_train = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_train_v3.csv', encoding='latin-1')
# data_dev = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_dev_v3.csv', encoding='latin-1')
# data_test = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_random_test_v3.csv', encoding='latin-1')
# data_test_fixed = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_fixed_test_v3.csv', encoding='latin-1')
def fix_doc_tgt(doc):
  new_doc = doc.replace('.[TGT]',' .\n[TGT] ')
  return new_doc

## if pl is true, append  paragraph leve data and label to the output
## if dl is true, append document leve data and label to the output
#### Both pl and dl can not be false
def load_paragraphs_documents(df,pl=True,dl=True):
    if not dl and not pl:
      print('both document level label and paragaph level label is false, choose one of them a True')
      return 
    labels = []
    texts = []
    num_doc = 0
    index = -1
    for doc in df['MASKED_DOCUMENT']:
      index += 1
      docs = doc.split('\n')
      doc_length = len(docs)

      if pd.isnull(df['Paragraph0'].iloc[index]):
      # add documents with no paragraph labels as one document and its label to the input data dataframe
        if dl:
          labels.append(df['TRUE_SENTIMENT'].iloc[index])
          texts.append(doc)
        num_doc +=1
        continue
      try:
        if  doc_length != 16 and pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):

          doc = fix_doc_tgt(doc)
          docs = doc.split('\n')
          doc_length = len(docs)
          if  doc_length != 16 and pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
            print('error on document %d'% df['DOCUMENT_INDEX'].iloc[index])
            print('document length is %s'%str(doc_length))
            continue
      except:
        print('this document has %d paragraphs %d' %(doc_length,df['DOCUMENT_INDEX'].iloc[index]))

      if pl:
        for i in range(doc_length):
          texts.append(docs[i])
          label_i = df['Paragraph%d'%i].iloc[index]
          labels.append(label_i)
        ### increase the effect of documents by adding the whole document per each paragraph :D
          if dl:
          ## add the document text and its label to the input data after adding each paragraph and their labels
            labels.append(df['TRUE_SENTIMENT'].iloc[index])
            texts.append(doc)
      else:
         if dl:
          ## add the document text and its label to the input data after adding each paragraph and their labels
            labels.append(df['TRUE_SENTIMENT'].iloc[index])
            texts.append(doc)
    print('number of one-paragraph docs: %d'%num_doc)
    return(texts,labels)
  

train_file = open('/content/train_v5.csv')
# train_file = open('/content/train_v4.csv')
train_df = pd.read_csv(train_file)

# dev_file = open('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_dev_v3.csv')
dev_file = open('/content/dev_v5.csv')
dev_df = pd.read_csv(dev_file)

# test_random_file = open('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_random_test_v3.csv')
test_random_file = open('/content/random_test_v5.csv')
test_random_df = pd.read_csv(test_random_file)

# test_fixed_file = open('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_fixed_test_v3.csv')
test_fixed_file = open('/content/fixed_test_v5.csv')
test_fixed_df = pd.read_csv(test_fixed_file)


# Load all files from a directory in a DataFrame.
def load_file(df,dl=True,pl=True):
  data = {}
  (texts,labels ) = load_paragraphs_documents(df,dl=dl,pl=pl)
  
  data["sentence"] = texts
  data["sentiment"] =labels
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(df,index = None,dl=True,pl=True):
  df_new = load_file(df,dl,pl)
  pos_df = df_new[df_new['sentiment'] == 'Positive']
  neg_df = df_new[df_new['sentiment'] == 'Negative']
  neu_df = df_new[df_new['sentiment'] == 'Neutral']
  pos_df["polarity"] = 1
  neg_df["polarity"] = -1
  neu_df["polarity"] = 0
  return pd.concat([pos_df, neg_df,neu_df]).sample(frac=1).reset_index(drop=True)

### train should consist both paragraph level and document level labels
train_all = load_dataset(train_df,dl=True,pl=True)
### two dev set, to test the model with paragraph level dev and document level dev
dev_par = load_dataset(dev_df,pl=True,dl=False)
dev_doc = load_dataset(dev_df,pl=False,dl=True)
### two random tests like above
test_par = load_dataset(test_random_df,pl=True,dl=False)
test_doc = load_dataset(test_random_df,pl=False,dl=True)
### two fixed tests like above
test_fixed_par = load_dataset(test_fixed_df,pl=True,dl=False)
test_fixed_doc = load_dataset(test_fixed_df,pl=False,dl=True)

print('Here the input consists of both paragraph level data and document level data')
print('Number of train inputs: %d\n Number of paragraph dev inputs: %d,Number of document level dev inputs: %d\n  \\
       Number of paragraph level fixed test inputs: %d, Number of document level fixed test inputs: %d \n \\
       Number of paragraph level random test inputs: %d,  Number of document level random test inputs: %d'
      %(len(train_all),len(dev_par),len(dev_doc),len(test_par),len(test_doc),len(test_fixed_par),len(test_fixed_doc)))


number of one-paragraph docs: 252


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

number of one-paragraph docs: 45
number of one-paragraph docs: 45
number of one-paragraph docs: 52
number of one-paragraph docs: 52
error on document 2171
document length is 6
number of one-paragraph docs: 114
error on document 2171
document length is 6
number of one-paragraph docs: 114
Here the input consists of both paragraph level data and document level data
Number of train inputs: 19178
 Number of paragraph dev inputs: 1881,Number of document level dev inputs: 284
  \       Number of paragraph level fixed test inputs: 1833, Number of document level fixed test inputs: 285 
 \       Number of paragraph level random test inputs: 1811,  Number of document level random test inputs: 339


In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
label_list = [-1, 0, 1]

In [0]:
train_InputExamples_all = train_all.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

dev_InputExamples_par = dev_par.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

dev_InputExamples_doc = dev_doc.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)




test_InputExamples_par = test_par.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
test_InputExamples_doc = test_doc.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples_fixed_par = test_fixed_par.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
test_InputExamples_fixed_doc = test_fixed_doc.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)



num_train_steps = int(len(train_InputExamples_all) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
# ITERATIONS_PER_LOOP = 1000 # I don't know what it is doing just decrease it to smaller value
ITERATIONS_PER_LOOP = int(len(train_InputExamples_all) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))



In [0]:
def create_model(is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels, bert_hub_module_handle):
  """Creates a classification model."""
  tags = set()
  if is_training:
    tags.add("train")
  bert_module = hub.Module(bert_hub_module_handle, tags=tags, trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use
  # bert_outputs["sequence_output"] instead.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu, bert_hub_module_handle):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,
        bert_hub_module_handle)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions)
        loss = tf.metrics.mean(per_example_loss)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions={"probabilities": probabilities})
    else:
      raise ValueError(
          "Only TRAIN, EVAL and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)



W0509 11:31:43.850744 140107810211712 estimator.py:1924] Estimator's model_fn (<function model_fn at 0x7f6d22ec7d70>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.6.48.82:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6d25089fd0>, '_model_dir': 'gs://bert_example/bert-tfhub/models/combined/weighted_loss/smallBERT-parDoc_256', '_protocol': None, '_save_checkpoints_steps': 200, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=599, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster': <tensorflow.python.distribute.cluster_resolver.tpu_cluster_resolver.TPU

I0509 11:31:43.854458 140107810211712 estimator.py:201] Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.6.48.82:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6d25089fd0>, '_model_dir': 'gs://bert_example/bert-tfhub/models/combined/weighted_loss/smallBERT-parDoc_256', '_protocol': None, '_save_checkpoints_steps': 200, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=599, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster': <tensorflow.python.distribute.c

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0509 11:31:43.857496 140107810211712 tpu_context.py:202] _TPUContext: eval_on_tpu True


In [0]:
# Train the model
def model_train(estimator):
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_InputExamples_all, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_InputExamples_all)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))



In [0]:
model_train(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 19178


I0509 11:31:43.885895 140107810211712 run_classifier.py:774] Writing example 0 of 19178


INFO:tensorflow:*** Example ***


I0509 11:31:43.917397 140107810211712 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Loss for final step: 0.0001550901.


I0509 11:42:00.349128 140107810211712 estimator.py:359] Loss for final step: 0.0001550901.


INFO:tensorflow:training_loop marked as finished


I0509 11:42:00.353009 140107810211712 error_handling.py:93] training_loop marked as finished


***** Finished training at 2019-05-09 11:42:00.361789 *****


#Evaluation and Prediction 

In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = dev_InputExamples_par#processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))

In [0]:
model_eval(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 1881


I0509 11:48:43.117894 140107810211712 run_classifier.py:774] Writing example 0 of 1881


INFO:tensorflow:*** Example ***


I0509 11:48:43.127058 140107810211712 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5990: gs://bert_example/bert-tfhub/models/combined/weighted_loss/smallBERT-parDoc_256/model.ckpt-5990


I0509 11:49:27.026344 140107810211712 estimator.py:2039] Saving 'checkpoint_path' summary for global step 5990: gs://bert_example/bert-tfhub/models/combined/weighted_loss/smallBERT-parDoc_256/model.ckpt-5990


INFO:tensorflow:evaluation_loop marked as finished


I0509 11:49:27.445385 140107810211712 error_handling.py:93] evaluation_loop marked as finished


***** Finished evaluation at 2019-05-09 11:49:27.447931 *****
***** Eval results *****
  eval_accuracy = 0.50265956
  eval_loss = 4.05732
  global_step = 5990
  loss = 4.0503583


In [0]:
#   eval_accuracy = 0.5473958
#   eval_loss = 1.1722633
#   global_step = 909
#   loss = 1.2637016
  
  
#   ***** Eval results *****
#   eval_accuracy = 0.5375
#   eval_loss = 1.2887536
#   global_step = 1212
#   loss = 1.3013718
def model_predict(estimator,prediction_examples):
  # Make predictions on a subset of eval examples
#   prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities']) for sentence, prediction in zip(prediction_examples, predictions)]



In [0]:
predictions = model_predict(estimator_from_tfhub,dev_InputExamples_par)

INFO:tensorflow:Writing example 0 of 1881


I0509 11:52:50.701092 140107810211712 run_classifier.py:774] Writing example 0 of 1881


INFO:tensorflow:*** Example ***


I0509 11:52:50.707783 140107810211712 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0509 11:53:25.232043 140107810211712 tpu_estimator.py:536] Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0509 11:53:25.234551 140107810211712 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0509 11:53:25.406697 140107810211712 error_handling.py:93] prediction_loop marked as finished


In [0]:
import numpy as np
from sklearn import metrics

labels = ["Negative","Neutral", "Positive"]
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(dev_par['sentiment'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))


[[ 29  70  47]
 [ 41 361 353]
 [ 44 381 555]]
              precision    recall  f1-score   support

    Negative       0.25      0.20      0.22       146
     Neutral       0.44      0.48      0.46       755
    Positive       0.58      0.57      0.57       980

   micro avg       0.50      0.50      0.50      1881
   macro avg       0.43      0.41      0.42      1881
weighted avg       0.50      0.50      0.50      1881



In [0]:
predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed_par)
import numpy as np
from sklearn import metrics
labels = ["Negative","Neutral", "Positive"]
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_fixed_par['sentiment'])




INFO:tensorflow:Writing example 0 of 1811


I0509 11:53:44.401680 140107810211712 run_classifier.py:774] Writing example 0 of 1811


INFO:tensorflow:*** Example ***


I0509 11:53:44.408467 140107810211712 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0509 11:54:17.255297 140107810211712 tpu_estimator.py:536] Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0509 11:54:17.259366 140107810211712 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0509 11:54:17.428230 140107810211712 error_handling.py:93] prediction_loop marked as finished


In [0]:
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

[[ 19 103 116]
 [ 43 337 347]
 [ 54 346 446]]
              precision    recall  f1-score   support

    Negative       0.16      0.08      0.11       238
     Neutral       0.43      0.46      0.45       727
    Positive       0.49      0.53      0.51       846

   micro avg       0.44      0.44      0.44      1811
   macro avg       0.36      0.36      0.35      1811
weighted avg       0.42      0.44      0.43      1811



In [0]:
predictions = model_predict(estimator_from_tfhub,test_InputExamples_par)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_par['sentiment'])


INFO:tensorflow:Writing example 0 of 1833


I0509 11:54:29.079323 140107810211712 run_classifier.py:774] Writing example 0 of 1833


INFO:tensorflow:*** Example ***


I0509 11:54:29.091821 140107810211712 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0509 11:55:02.962666 140107810211712 tpu_estimator.py:536] Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0509 11:55:02.965048 140107810211712 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0509 11:55:03.141522 140107810211712 error_handling.py:93] prediction_loop marked as finished


In [0]:
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

[[ 20  85  62]
 [ 42 348 299]
 [ 34 396 547]]
              precision    recall  f1-score   support

    Negative       0.21      0.12      0.15       167
     Neutral       0.42      0.51      0.46       689
    Positive       0.60      0.56      0.58       977

   micro avg       0.50      0.50      0.50      1833
   macro avg       0.41      0.39      0.40      1833
weighted avg       0.50      0.50      0.50      1833



In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)

print('-------- Document level Dev Result---------')
predictions = model_predict(estimator_from_tfhub,dev_InputExamples_doc)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(dev_doc['sentiment'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

print('-------- Document level Fixed Test Result---------')
predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed_doc)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_fixed_doc['sentiment'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

print('-------- Document level Random Test Result---------')
predictions = model_predict(estimator_from_tfhub,test_InputExamples_doc)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_doc['sentiment'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

-------- Document level Dev Result---------
[[  2  11  10]
 [  0  47  50]
 [  2  46 116]]
              precision    recall  f1-score   support

    Negative       0.50      0.09      0.15        23
     Neutral       0.45      0.48      0.47        97
    Positive       0.66      0.71      0.68       164

   micro avg       0.58      0.58      0.58       284
   macro avg       0.54      0.43      0.43       284
weighted avg       0.58      0.58      0.57       284

-------- Document level Fixed Test Result---------
[[  2  27  21]
 [  4  49  63]
 [  2  50 121]]
              precision    recall  f1-score   support

    Negative       0.25      0.04      0.07        50
     Neutral       0.39      0.42      0.40       116
    Positive       0.59      0.70      0.64       173

   micro avg       0.51      0.51      0.51       339
   macro avg       0.41      0.39      0.37       339
weighted avg       0.47      0.51      0.48       339

-------- Document level Random Test Result---------

In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)

print('-------- Document level Dev Result---------')
predictions = model_predict(estimator_from_tfhub,dev_InputExamples_doc)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(dev_doc['sentiment'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

print('-------- Document level Fixed Test Result---------')
predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed_doc)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_fixed_doc['sentiment'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

print('-------- Document level Random Test Result---------')
predictions = model_predict(estimator_from_tfhub,test_InputExamples_doc)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_doc['sentiment'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

-------- Document level Dev Result---------
[[  2  15   6]
 [  0  48  49]
 [  2  52 110]]
              precision    recall  f1-score   support

    Negative       0.50      0.09      0.15        23
     Neutral       0.42      0.49      0.45        97
    Positive       0.67      0.67      0.67       164

   micro avg       0.56      0.56      0.56       284
   macro avg       0.53      0.42      0.42       284
weighted avg       0.57      0.56      0.55       284

-------- Document level Fixed Test Result---------
[[  3  28  19]
 [  5  53  58]
 [  2  60 111]]
              precision    recall  f1-score   support

    Negative       0.30      0.06      0.10        50
     Neutral       0.38      0.46      0.41       116
    Positive       0.59      0.64      0.61       173

   micro avg       0.49      0.49      0.49       339
   macro avg       0.42      0.39      0.38       339
weighted avg       0.47      0.49      0.47       339

-------- Document level Random Test Result---------

In [0]:
mds = []
evs = []
pds = []
tf.logging.set_verbosity(tf.logging.FATAL) 
for i in range(1,20):
  print('----------------------- Starting Epoch %d-----------------------'%i)

  NUM_TRAIN_EPOCHS = i
  
  num_train_steps = int(len(train_InputExamples_all) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
  num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

#   model_fn = model_fn_builder(
#   num_labels=len(label_list),
#   learning_rate=LEARNING_RATE,
#   num_train_steps=num_train_steps,
#   num_warmup_steps=num_warmup_steps,
#   use_tpu=True,
#   bert_hub_module_handle=BERT_MODEL_HUB)


  model_fn = model_fn_builder(
    num_labels=len(label_list),
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=True,
    bert_hub_module_handle=BERT_MODEL_HUB
  )

  estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=model_fn,
    config=get_run_config(OUTPUT_DIR),
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=PREDICT_BATCH_SIZE,
  )


  
  md = model_train(estimator_from_tfhub)
  
  
  print('-------- Document level Dev Result---------')
  predictions = model_predict(estimator_from_tfhub,dev_InputExamples_doc)
  labels_val = []
  for item in predictions:
    labels_val.append(labels[np.argmax(item[1])])
  true_label = list(dev_doc['sentiment'])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  
  
  print('-------- Paragraph level Dev Result---------')
  predictions = model_predict(estimator_from_tfhub,dev_InputExamples_par)
  labels_val = []
  for item in predictions:
    labels_val.append(labels[np.argmax(item[1])])
  true_label = list(dev_par['sentiment'])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

  print('-------- Document level Fixed Test Result---------')
  predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed_doc)
  labels_val = []
  for item in predictions:
    labels_val.append(labels[np.argmax(item[1])])
  true_label = list(test_fixed_doc['sentiment'])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  
  
  print('-------- Paragraph level Fixed Test Result---------')
  predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed_par)
  labels_val = []
  for item in predictions:
    labels_val.append(labels[np.argmax(item[1])])
  true_label = list(test_fixed_par['sentiment'])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

  print('-------- Document level Random Test Result---------')
  predictions = model_predict(estimator_from_tfhub,test_InputExamples_doc)
  labels_val = []
  for item in predictions:
    labels_val.append(labels[np.argmax(item[1])])
  true_label = list(test_doc['sentiment'])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  
  print('-------- Paragraph level Random Test Result---------')
  predictions = model_predict(estimator_from_tfhub,test_InputExamples_par)
  labels_val = []
  for item in predictions:
    labels_val.append(labels[np.argmax(item[1])])
  true_label = list(test_par['sentiment'])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  
  


----------------------- Starting Epoch 1-----------------------
***** Started training at 2019-05-09 12:07:13.408352 *****
  Num examples = 19178
  Batch size = 32
***** Finished training at 2019-05-09 12:11:33.467409 *****
-------- Document level Dev Result---------
[[  0  17   6]
 [  1  37  59]
 [  2  36 126]]
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        23
     Neutral       0.41      0.38      0.40        97
    Positive       0.66      0.77      0.71       164

   micro avg       0.57      0.57      0.57       284
   macro avg       0.36      0.38      0.37       284
weighted avg       0.52      0.57      0.55       284

-------- Paragraph level Dev Result---------
[[  1  92  53]
 [  1 317 437]
 [  4 244 732]]
              precision    recall  f1-score   support

    Negative       0.17      0.01      0.01       146
     Neutral       0.49      0.42      0.45       755
    Positive       0.60      0.75      0.66       9